In [ ]:
from IPython.display import display, HTML

REQUIRED_VERSIONS = {
    "hbp_service_client": "1.1.0",
    'ipywidgets':'7.0.5'
}

def get_version(package_name):
    """Returns version for the given package"""
    if package_name=='hbp_service_client':
        import hbp_service_client
        return hbp_service_client.__version__
    if package_name=='ipywidgets':
        import ipywidgets
        return '.'.join(map(str, ipywidgets.version_info[0:3]))
    
def install_package(package_name, versions):
    """Updates the specified package."""
    print("Changing package {}".format(package_name))
    if package_name=='hbp_service_client':
        lib = "{}=={}".format(package_name, versions[package_name])
        !pip install -I --quiet $lib     
    if package_name=='ipywidgets':
        lib1 = 'ipywidgets=={}'.format(versions["ipywidgets"])
        versions["widgetsnbextension"]='3.0.8'
        lib2 = 'widgetsnbextension=={}'.format(versions["widgetsnbextension"])
        !pip2 install $lib1 $lib2
        !pip3 install $lib1 $lib2
        !jupyter nbextension uninstall --py widgetsnbextension
        !jupyter nbextension install --py --user widgetsnbextension
        !jupyter nbextension enable widgetsnbextension --user --py
    
# Check if a package needs to be changed
reload_required = []
for package_name, required_version in REQUIRED_VERSIONS.items():
    if get_version(package_name) != required_version:
        print("{} {}".format(get_version(package_name), required_version))
        reload_required.append(package_name)
        
if reload_required:
    display(HTML("""
    <div class="alert alert-block alert-danger">Some internal libraries had to be changed...<br>
    Please follow the popup to reload this page.</div>"""))
    
    # Change all package that requires change
    for package in reload_required:
        install_package(package, REQUIRED_VERSIONS)
    
    display(HTML("""<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
    Jupyter.notebook.dirty = false; window.location.reload(); })</script>"""))
else:
    display(HTML("""
    <div class="alert alert-block alert-success">Setup for this notebook is correct.</div>"""))

In [ ]:
from ipywidgets import widgets
from IPython.display import *
from ipywidgets import *
from IPython.display import HTML,Javascript, display
import pprint
import os
import shutil
os.chdir("/home/jovyan")
if os.path.isdir('GUIBPOPT')==True:
    shutil.rmtree('GUIBPOPT')
if os.path.isdir('GUIBPOPT')==False:
    !mkdir GUIBPOPT
os.chdir("GUIBPOPT")
from IPython.display import HTML, Javascript, display
display(HTML('''<style>
    .widget-label { min-width: 17ex !important;}
</style>'''))

javascript2 = """
<script type="text/Javascript">
document.styleSheets[0].insertRule(".widget-listbox.form-control { width: auto }", 1);
document.styleSheets[0].insertRule(".form_login > label { width: 30% }", 1);
document.styleSheets[0].insertRule(".form_login > label + input { width: 70% }", 1);
</script>
"""
h1=HTML(javascript2)
h1.display='none'
#display(h1)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%capture
URL=('https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/'
'hippocampus_optimization/'
'bsp_data_repository-4.0.5_Python3.tar.gz')
!wget "{URL}"
!tar -zxvf bsp_data_repository-4.0.5_Python3.tar.gz

In [ ]:
clients = get_hbp_service_client()
import zipfile
listfilenames=[]
for filename in os.listdir(os.getcwd()):
    listfilenames.append(filename)
if 'Work-in-progress3.png' not in listfilenames:
    clients.storage.download_file(os.path.join(os.path.sep,'9132',\
                                               'Work-in-progress3.png'), 'Work-in-progress3.png')
if 'unicore_api_Python3.ipynb' not in listfilenames:
    clients.storage.download_file(os.path.join(os.path.sep,'9130','unicore_api_Python3.ipynb'), 'unicore_api_Python3.ipynb')
diff = lambda l1,l2: [x for x in l1 if x not in l2]
os.chdir('bsp_data_repository-4.0.5_Python3')
os.chdir('optimizations')
FD=diff(os.listdir('.'),['README'])
for i in range(len(FD)):
    shutil.copytree(FD[i],os.path.join('..','..',FD[i]))
import re
F1=[]
F2=[]
for i in range(len(FD)):
    poz=[m.start() for m in re.finditer('_', FD[i])]
    F2.append(FD[i][poz[0]+1:poz[1]])
    F1.append(FD[i][poz[2]+1:poz[len(poz)-1]])
os.chdir('..')
os.chdir('..')

In [ ]:
import logging
import unicodedata
from dateutil import parser
import pytz
%run "unicore_api_Python3.ipynb"
logger = logging.getLogger()
logging.disable(logging.WARNING)

In [ ]:
from ipywidgets import widgets
from IPython.display import *
from ipywidgets import *
import os
import pandas 
import json
import io
import base64
import datetime
def plot_to_html(plt):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return """<img src='data:image/png;base64,{}'/>""".format(base64.b64encode(buf.getvalue()).decode('ascii'))

F3=[]
for i in range(len(F1)):
    F3.append(F1[i]+' ('+F2[i]+')')

w3=Select(description='morphology', options=set(F3),layout=Layout(width='50%',height='100px'))
display(w3)

In [ ]:
indices = [i for i, x in enumerate(F1) if x == w3.value[0:w3.value.find(' ')]]

w4=Select(description='configuration',layout=Layout(width='50%',height='100px'))
FF=[]
for i in range(len(indices)):
    FF.append(FD[indices[i]])
w4.options=sorted(FF)

for files in os.listdir(w4.value):
    if files.endswith('.zip'):
        os.chdir(w4.value)
        zip_ref = zipfile.ZipFile(files, 'r')
        zip_ref.extractall()
        zip_ref.close()          
os.chdir('..')
        
import neurom as nm
os.chdir(w4.value)
os.chdir(w4.value)
os.chdir('morphology')
for files in os.listdir('.'):
    if files.endswith('.asc'):
        nrn = nm.load_neuron(w3.value[0:w3.value.find(' ')]+'.asc');
    else:
        if files.endswith('.ASC'):
            nrn = nm.load_neuron(w3.value[0:w3.value.find(' ')]+'.ASC');
from neurom import viewer
import matplotlib
import numpy as np
%matplotlib agg
plt,ax=viewer.draw(nrn, mode='2d');
plt.savefig("{}.png".format(w3.value))
plt.clear()
%matplotlib inline
file = open("{}.png".format(w3.value), "rb")
image = file.read()
out1=widgets.Image(value=image,format='png',width=400,height=500,)
display(out1)
os.chdir('..')
os.chdir('..')
os.chdir('..')

H1=widgets.Textarea(layout=Layout(width='80%', height='200px'))
H2=widgets.Textarea(layout=Layout(width='80%', height='200px'))
os.chdir(w4.value)
os.chdir(w4.value)
os.chdir('config')
with open('features.json') as data_file:    
    data = json.load(data_file)
H1.value=pprint.pformat(data[list(data.keys())[0]],width=80,depth=5)
H1.disabled=True
with open('parameters.json') as data_file:    
    data = json.load(data_file)
H2.value=pprint.pformat(data,width=120,depth=5)
os.chdir('..')
os.chdir('..')
os.chdir('..')
BT1 = Button()
BT1.description = 'Save new parameters'
BT1.background_color='gainsboro'
BT1.border_color='black'
import ast
def saveparamfile(_):
    os.chdir(w4.value)
    os.chdir(w4.value)
    os.chdir('config')
    with open('parameters.json','w') as f:
        json.dump(ast.literal_eval(H2.value),f)
    f.close()
    os.chdir('..')
    os.chdir('..')
    os.chdir('..')
BT1.on_click(saveparamfile)
page1=widgets.Box(children=[w4])
page2=widgets.Box(children=[H1])
page3=widgets.Box(children=[widgets.VBox([H2,BT1],layout=Layout(width='100%'))])
tabs = widgets.Tab(children=[page1,page2,page3])
tabs.set_title(0, 'Configuration')
tabs.set_title(1, 'Features')
tabs.set_title(2, 'Parameters')
display(tabs)

def on_w3_clicked(change):
    out1.layout.display='none'
    %matplotlib agg
    clear_output(wait=True)
    w4.layout.display='none'
    tabs.layout.display='none'
    indices = [i for i, x in enumerate(F1) if x == w3.value[0:w3.value.find(' ')]]
    FF=[]
    for i in range(len(indices)):
        FF.append(FD[indices[i]])
    dirnew=sorted(FF)[0]
    for files in os.listdir(dirnew):
        if files.endswith('.zip'):
            os.chdir(dirnew)
            zip_ref = zipfile.ZipFile(files, 'r')
            zip_ref.extractall()
            zip_ref.close()          
    os.chdir('..')
    os.chdir(dirnew)
    os.chdir(dirnew)
    os.chdir('morphology')
    for files in os.listdir('.'):
        if files.endswith('.asc'):
            nrn = nm.load_neuron(w3.value[0:w3.value.find(' ')]+'.asc');
        else:
            if files.endswith('.ASC'):
                nrn = nm.load_neuron(w3.value[0:w3.value.find(' ')]+'.ASC');
    plt,ax=viewer.draw(nrn, mode='2d');
    for files in os.listdir('.'):
        if files.endswith('.png'):
            os.remove(files)
    plt.savefig("{}.png".format(w3.value))
    plt.clear()
    %matplotlib inline
    file = open("{}.png".format(w3.value), "rb")
    image = file.read()
    out1.value=image
    out1.layout.display=''
    os.chdir('..')
    os.chdir('config')
    with open('features.json') as data_file:    
        data = json.load(data_file)
    H1.value=pprint.pformat(data[list(data.keys())[0]],width=80,depth=5)
    H1.disabled=True
    with open('parameters.json') as data_file:    
        data = json.load(data_file)
    H2.value=pprint.pformat(data,width=120,depth=5)
    os.chdir('..')
    os.chdir('..')
    os.chdir('..')
    w4.options=sorted(FF)
    w4.layout.display=''
    tabs.selected_index=0
    tabs.layout.display=''
    
def on_w4_clicked(change):
    clear_output(wait=True)
    os.chdir(w4.value)
    if os.path.isdir(w4.value)==False:
        dirloc=os.getcwd()
        for files in os.listdir(dirloc):
            if files.endswith('.zip'):
                os.chdir(dirloc)
                zip_ref = zipfile.ZipFile(files, 'r')
                zip_ref.extractall()
                zip_ref.close()  
    os.chdir(w4.value)
    os.chdir('config')
    with open('features.json') as data_file:    
        data = json.load(data_file)
    H1.value=pprint.pformat(data[list(data.keys())[0]],width=80,depth=5)
    H1.disabled=True
    with open('parameters.json') as data_file:    
        data = json.load(data_file)
    H2.value=pprint.pformat(data,width=120,depth=5)
    os.chdir('..')
    os.chdir('..')
    os.chdir('..')
w3.observe(on_w3_clicked,names='value')
w4.observe(on_w4_clicked,names='value')

NGEN=Text(**dict(
        margin='0px 0px 0px 16px'
))
NGEN.description='max_ngen'
NGEN.value='2'
NGEN.width=600
display(NGEN)
OS=Text(**dict(
        margin='0px 0px 0px 16px'
))
OS.description='offspring_size'
OS.value='10'
OS.width=200
display(OS)
zfName=''
import zipfile
from shutil import copy2
def createzip(_):
    os.chdir(w4.value)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith('init') or file.endswith('.zip'):
            os.remove(file)
    with open('init.py','w') as f:
        f.write('import os')
        f.write('\n')
        f.write('os.system(\'python opt_neuron.py --max_ngen='+NGEN.value+' --offspring_size='+OS.value+\
                ' --start --checkpoint ./checkpoints/checkpoint.pkl\')')
        f.write('\n')
    f.close()
    os.chdir(w4.value)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith('init'):
            os.remove(file)
    os.chdir('..')
    shutil.copy('init.py',os.path.join('.',w4.value,'init.py'))
    global zfName
    poz=[m.start() for m in re.finditer('_', w4.value)]
    zfName = w4.value[0:poz[len(poz)-1]]+'_'+datetime.datetime.now().strftime('%Y%m%d%H%M%S')+'.zip'
    shutil.copytree(w4.value,zfName[0:zfName.find('.zip')])
    foldnameinside=w4.value #zfName[0:zfName.find('.zip')]
    foo = zipfile.ZipFile(zfName, 'w')
    for root, dirs, files in os.walk(os.path.join('.',foldnameinside)):
        if (root==os.path.join('.',foldnameinside,'morphology')) or \
        (root==os.path.join('.',foldnameinside,'config')) or \
        (root==os.path.join('.',foldnameinside,'mechanisms')) or \
        (root==os.path.join('.',foldnameinside,'model')):
            for f in files:
                foo.write(os.path.join(root, f))
        if (root==os.path.join('.',foldnameinside,'checkpoints')) or \
        (root==os.path.join('.',foldnameinside,'figures')):
            foo.write(os.path.join(root))
        if (root==os.path.join('.',foldnameinside)):
            for f in files:
                if f.endswith('.py'):
                    foo.write(os.path.join(root, f))                    
    foo.close()
    os.chdir('..')
    BT.disabled=True
    
def createzipPizDaint(_):
    os.chdir(w4.value)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith("ipyparallel.sbatch") or file.startswith("zipfolder.py"):
            os.remove(file)
    with open('zipfolder.py','w') as f:
        f.write('import os\n')
        f.write('import zipfile\n')
        f.write('retval = os.getcwd()\n')
        f.write('print("Current working directory %s" % retval)\n')
        f.write('os.chdir(\'..\')\n')
        f.write('retval = os.getcwd()\n')
        f.write('print("Current working directory %s" % retval)\n')
        f.write('def zipdir(path, ziph):\n')
        f.write('    for root, dirs, files in os.walk(path):\n')
        f.write('        for file in files:\n')
        f.write('            ziph.write(os.path.join(root, file))\n')
        f.write('zipf = zipfile.ZipFile(\'output.zip\', \'w\')\n')
        f.write('zipdir(\''+w4.value+'/\', zipf)\n')
    f.close()
    with open('ipyparallel.sbatch','w') as f:
        f.write('#!/bin/bash -l\n')
        f.write('\n')
        f.write('mkdir logs\n')
        f.write('#SBATCH --job-name=bluepyopt_ipyparallel\n')
        f.write('#SBATCH --error=logs/ipyparallel_%j.log\n')
        f.write('#SBATCH --output=logs/ipyparallel_%j.log\n')
        f.write('#SBATCH --partition=normal\n')
        f.write('#SBATCH --constraint=mc\n')
        f.write('\n')
        f.write('export PMI_NO_FORK=1\n')
        f.write('export PMI_NO_PREINITIALIZE=1\n')
        f.write('export PMI_MMAP_SYNC_WAIT_TIME=300 \n')
        f.write('set -e\n')
        f.write('set -x\n')
        f.write('\n')
        f.write('export MODULEPATH=/users/bp000178/ich002/software/daint/'+\
                'local-20191210122932/share/modules:$MODULEPATH;module load bpopt\n')
        f.write('\n')
        f.write('export USEIPYP=1\n')
        f.write('export IPYTHONDIR="`pwd`/.ipython"\n')
        f.write('export IPYTHON_PROFILE=ipyparallel.${SLURM_JOBID}\n')
        f.write('ipcontroller --init --sqlitedb --ip=\'*\' --profile=${IPYTHON_PROFILE} &\n')
        f.write('sleep 30\n')
        f.write('srun ipengine --profile=${IPYTHON_PROFILE} &\n')
        f.write('CHECKPOINTS_DIR="checkpoints"\n')
        f.write('nrnivmodl mechanisms\n')
        f.write('python opt_neuron.py --offspring_size='+OS.value+\
                ' --max_ngen='+NGEN.value+' --start --checkpoint "${CHECKPOINTS_DIR}/checkpoint.pkl"\n') 
        f.write('python zipfolder.py')
        f.write('\n')
    f.close()
    os.chdir(w4.value)
    dirloc=os.getcwd()
    for file in os.listdir(dirloc):
        if file.startswith('ipyparallel.sbatch') or file.startswith("zipfolder.py"):
            os.remove(file)
    os.chdir('..')
    shutil.copy('ipyparallel.sbatch',os.path.join('.',w4.value,'ipyparallel.sbatch'))
    shutil.copy('zipfolder.py',os.path.join('.',w4.value,'zipfolder.py'))
    global zfNamePiz
    poz=[m.start() for m in re.finditer('_', w4.value)]
    zfNamePiz = w4.value[0:poz[len(poz)-1]]+'_'+datetime.datetime.now().strftime('%Y%m%d%H%M%S')+\
    '.zip'
    shutil.copytree(w4.value,zfNamePiz[0:zfNamePiz.find('.zip')])
    foldnameinside=w4.value #zfNamePiz[0:zfNamePiz.find('.zip')]
    foo = zipfile.ZipFile(zfNamePiz, 'w')
    for root, dirs, files in os.walk(os.path.join('.',foldnameinside)):
        if (root==os.path.join('.',foldnameinside,'morphology')) or \
        (root==os.path.join('.',foldnameinside,'config')) or \
        (root==os.path.join('.',foldnameinside,'mechanisms')) or \
        (root==os.path.join('.',foldnameinside,'model')):
            for f in files:
                foo.write(os.path.join(root, f))
        if (root==os.path.join('.',foldnameinside,'checkpoints')) or \
        (root==os.path.join('.',foldnameinside,'figures')):
            foo.write(os.path.join(root))
        if (root==os.path.join('.',foldnameinside)):
            for f in files:
                if f.endswith('.py') or f.endswith('.sbatch'):
                    foo.write(os.path.join(root, f))                    
    foo.close()
    os.chdir('..')
    H2.disabled=True
    BT1.disabled=True
    NGEN.disabled=True
    OS.disabled=True
    BTCSCS.disabled=True
    
BT = Button(description = 'Create zip file', layout=Layout(width='40%'))
BT.background_color='gainsboro'
BT.border_color='black'
BT.width='20%'
BT.on_click(createzip)
BTCSCS = Button(description = 'Create Piz Daint zip file', layout=Layout(width='40%'))
BTCSCS.background_color='gainsboro'
BTCSCS.border_color='black'
BTCSCS.width='20%'
BTCSCS.on_click(createzipPizDaint)

import logging
logging.captureWarnings(False)

In [ ]:
from IPython.display import HTML,Javascript, display
from ipywidgets import widgets
import unicodedata

username=''

def checkloginNSG(_):
    username=username_widget.value
    password=password_widget.value
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    headers = {'cipres-appkey' : KEY}
    URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/'+username
    r = requests.get(URL, auth=(username, password),headers=headers)
    root = xml.etree.ElementTree.fromstring(r.text)
    a=0
    for child in root:
        if child.tag == 'displayMessage':
            a=1
            msg=child.text
    if a==0:
        msg='Authenticated successfully'
        msg_widget.disabled=True
        buttonlogin.disabled=True
        username_widget.disabled=True
        password_widget.disabled=True
    msg_widget.value=msg
    msg_widget.layout.display=''

username_widget = widgets.Text(description='Username:')
username_widget.layout.width='37.5%'
password_widget = widgets.Password(description='Password:')
buttonlogin = Button()
buttonlogin.description = 'Login NSG'
buttonlogin.background_color='gainsboro'
buttonlogin.border_color='black'
buttonlogin.width='20%'
buttonlogin.on_click(checkloginNSG)
msg_widget = widgets.Text()
msg_widget.layout.width='40%'
msg_widget.layout.display='none'
formMNSG=widgets.VBox(children=[username_widget,password_widget,buttonlogin,msg_widget])
formMNSG.layout.display=''

NNODES=Text()
NNODES.description='number nodes'
NNODES.value='2'
NNODES.width=190
NCORES=Text()
NCORES.description='number cores'
NCORES.value='10'
NCORES.width=190
RT=Text()
RT.description='runtime'
RT.value='1'
RT.width=190
IN=widgets.HTML(value='<img src="./GUIBPOPT/Work-in-progress3.png", width=400>')
IN.layout.display='none'
STN=Text()
STN.description='status'
STN.value=''
STN.disabled=True
STN.width=75
STN.layout.display='none'

TT=Text()
TT.description='Title:'
NNODESCSCS=Text()
NNODESCSCS.description='number nodes'
NNODESCSCS.value='6'
NNODESCSCS.width=190
NCORESCSCS=Text()
NCORESCSCS.description='number cores'
NCORESCSCS.value='24'
NCORESCSCS.width=190
RTCSCS=Text()
RTCSCS.description='runtime'
RTCSCS.value='120m'
RTCSCS.width=190
INCSCS=widgets.HTML(value='<img src="./GUIBPOPT/Work-in-progress3.png", width=400>')
INCSCS.layout.display='none'
STNCSCS=Text()
STNCSCS.description='status'
STNCSCS.value=''
STNCSCS.disabled=True
STNCSCS.width=75
STNCSCS.layout.display='none'

import os
import requests
import xml.etree.ElementTree
import time
import sys

def runNSG(_):
    myboxUNSG.children[0].disabled=True
    IN.layout.display=''
    os.chdir(w4.value)
    
    global CRA_USER, PASSWORD, KEY, URL, TOOL
    username=username_widget.value
    password=password_widget.value
    CRA_USER = username
    PASSWORD = password
    KEY = 'Application_Fitting-DA5A3D2F8B9B4A5D964D4D2285A49C57'
    URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1'
    TOOL = 'BLUEPYOPT_TG'
    
    os.popen('pwd').read()
    os.popen('ls -ltr').read()
    headers = {'cipres-appkey' : KEY}
    payload = {'tool' : TOOL, 'metadata.statusEmail' : 'false', 'vparam.number_cores_' : \
               unicodedata.normalize('NFKD', NCORES.value).encode('ascii','ignore'), \
               'vparam.number_nodes_' : unicodedata.normalize('NFKD', NNODES.value).encode('ascii','ignore'), \
               'vparam.runtime_' : unicodedata.normalize('NFKD', RT.value).encode('ascii','ignore'), \
               'vparam.filename_': 'init.py'}
    files = {'input.infile_' : open(zfName,'rb')}

    r = requests.post('{}/job/{}'.format(URL, CRA_USER), auth=(CRA_USER, PASSWORD), data=payload, \
                      headers=headers, files=files)
    
    root = xml.etree.ElementTree.fromstring(r.text)

    global outputuri, selfuri
    for child in root:
        if child.tag == 'resultsUri':
            for urlchild in child:
                if urlchild.tag == 'url':
                    outputuri = urlchild.text
        if child.tag == 'selfUri':
            for urlchild in child:
                if urlchild.tag == 'url':
                    selfuri = urlchild.text

    r = requests.get(selfuri, auth=(CRA_USER, PASSWORD), headers=headers)
    root = xml.etree.ElementTree.fromstring(r.text)
    for child in root:
        if child.tag=='jobHandle':
            jobname=child.text
    clients = get_hbp_service_client()
    hdr = oauth.get_token()
    my_url = 'https://services.humanbrainproject.eu/idm/v1/api/user/me'
    headers2 = {'Authorization':'Bearer ' + hdr}
    res = requests.get(my_url, headers = headers2)
    filetoupload=str("listjobsNSG_"+res.json()['username']+".txt")
    if clients.storage.exists(os.path.join(get_collab_storage_path(),filetoupload)):
        clients.storage.download_file(os.path.join(get_collab_storage_path(),filetoupload), filetoupload);
        clients.storage.delete(os.path.join(get_collab_storage_path(),filetoupload))
    with open("listjobsNSG_"+res.json()['username']+".txt", "a") as myfile:
        myfile.write(jobname)
        myfile.write(' ')
        myfile.write(zfName[0:zfName.find('.zip')])
        myfile.write('\n')
    myfile.close()
    clients.storage.upload_file(filetoupload,os.path.join(get_collab_storage_path(),filetoupload),'txt')
    
    myboxUNSG.children[1].layout.display=''
    os.chdir('..')
    
def runNSGifusernameandconfig(_):
    username=username_widget.value
    password=password_widget.value
    if username!='':
        if zfName!='':
            runNSG(_)
        else:
            js = "<script>alert('You have to create the zip file first');</script>"
            display(HTML(js)) 
    else:
        js = "<script>alert('You have to login NSG first');</script>"
        display(HTML(js)) 
    
import datetime
    
import re
import tarfile
def checksimnsg(CheckSNSG):
    os.chdir(w4.value)
    headers = {'cipres-appkey' : KEY}
    r = requests.get(selfuri, auth=(CRA_USER, PASSWORD), headers=headers)
    root = xml.etree.ElementTree.fromstring(r.text)
    for child in root:
        if child.tag == 'jobStage':
            jobtodisp=child.text
        if child.tag == 'terminalStage':
            jobstatus = child.text
    STN.layout.display=''
    STN.value=jobtodisp
    if jobstatus=='true':
        r = requests.get(outputuri,headers= headers, auth=(CRA_USER, PASSWORD))
        globaldownloadurilist = []
        root = xml.etree.ElementTree.fromstring(r.text)
        for child in root:
            if child.tag == 'jobfiles':
                for jobchild in child:
                    if jobchild.tag == 'jobfile':
                        for downloadchild in jobchild:
                            if downloadchild.tag == 'downloadUri':
                                for attchild in downloadchild:
                                    if attchild.tag == 'url':
                                        globaldownloadurilist.append(attchild.text)
        # download all output files
        for downloaduri in globaldownloadurilist:
            r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
            d = r.headers['content-disposition']
            filename_list = re.findall('filename=(.+)', d)
            for filename in filename_list:
                with open(filename, 'wb') as fd:
                    for chunk in r.iter_content():
                        fd.write(chunk) 
        
        clients = get_hbp_service_client()
        storeto_path1 = os.path.join(get_collab_storage_path(),"BluePyOptAll")
        if not clients.storage.exists(storeto_path1):
            clients.storage.mkdir(storeto_path1)
        # want a unique dir for each job run
        storeto_path2 = os.path.join(storeto_path1,"resultsNSG")
        if not clients.storage.exists(storeto_path2):
            clients.storage.mkdir(storeto_path2)
        hdr = get_bbp_client().task.oauth_client.get_auth_header()
        my_url = 'https://services.humanbrainproject.eu/idm/v1/api/user/me'
        headers2 = {'Authorization':hdr}
        res = requests.get(my_url, headers = headers2)
        storeto_path3 = str(os.path.join(storeto_path2,res.json()['username']))
        if not clients.storage.exists(storeto_path3):
            clients.storage.mkdir(storeto_path3)        
        storeto = os.path.join(storeto_path3,zfName[0:zfName.find('.zip')])
        clients.storage.mkdir(storeto)
        for files in os.listdir('.'):
            if files.startswith('STDOUT') or files.startswith('STDERR'):
                clients.storage.upload_file(files,os.path.join(storeto,files),'txt')
                os.remove(files)
            if files.startswith('scheduler'):
                clients.storage.upload_file(files,os.path.join(storeto,files),'txt')
                os.remove(files)
            if files.endswith('.tar.gz'):
                clients.storage.upload_file(files,os.path.join(storeto,files),'tar')
                os.remove(files)
              
        
        STN.layout.display='none'
        IN.layout.display='none'
        CheckSNSG.layout.display='none'
        runningUNSG=False
        myboxUNSG.children[0].disabled=False
    os.chdir('..')
    
def rununicoreCSCS(_):
    RunSCSCS.disabled=True
    INCSCS.layout.display=''
    
    os.chdir(w4.value)
    with open(zfNamePiz, 'rb') as content_file:
        content = content_file.read()
    fjob1 = {'To': zfNamePiz, 'Data':content}
    inputs = [fjob1]
    if TT.value=='':
        jobname='UNICORE_Job'
    else:
        jobname=TT.value
    job = {}
    job = {"Executable": "unzip "+zfNamePiz+"; cd "+w4.value+"; chmod +rx *.sbatch; ./ipyparallel.sbatch",
           "Name": jobname,
           "Resources": {"Nodes": NNODESCSCS.value,\
                         "CPUsPerNode": NCORESCSCS.value, \
                         "Runtime": RTCSCS.value,\
                         #"Memory": "4G",\
                         "NodeConstraints" : "mc",\
                        }, 
          }
    auth = get_oidc_auth() 
    base_url = get_sites()['DAINT-CSCS']['url']
    global job_url
    job_url = submit(base_url + '/jobs', job, auth, inputs)
    if job_url!='':
        myboxUCSCS.children[1].layout.display=''
    else:
        INCSCS.layout.display='none'
        myboxUCSCS.children[0].disabled=False
    os.chdir('..')
        
def assign_file_types(filename):
    """ assign file types based on file name and extension """
    fname, extension = os.path.splitext(filename)
    if extension == '.txt':
        filetype = 'text/plain'
    else:
        filetype = 'application/unknown'
    return filetype

def checksimCSCS(CheckCSCS):
    RunSCSCS.disabled=True
    auth = get_oidc_auth() 
    job_properties = get_properties(job_url, auth)
    STNCSCS.layout.display=''
    STNCSCS.value=job_properties['status']
    if STNCSCS.value=='SUCCESSFUL':
        auth = get_oidc_auth()
        wd = get_working_directory(job_url, auth)
        output_files = list_files(wd, auth)
        clients = get_hbp_service_client()
        storeto_path = os.path.join(get_collab_storage_path(),'results')
        if not clients.storage.exists(storeto_path):
            clients.storage.mkdir(storeto_path)
        storeto = str(os.path.join(storeto_path,'output_'+job_properties['submissionTime']))
        clients.storage.mkdir(storeto)
        for file_path in output_files:
            _, f = os.path.split(file_path)
            if (f=='stderr') or (f=='stdout') or (f=='output.zip'):
                filetype = assign_file_types(f)
                content = get_file_content(wd+"/files"+file_path,auth)
                with open(f,"wb") as local_file:
                    local_file.write(content)
                clients.storage.upload_file(str(f),str(os.path.join(storeto,f)),filetype)
        STNCSCS.layout.display='none'
        INCSCS.layout.display='none'
        CheckSCSCS.layout.display='none'
        myboxUCSCS.children[1].layout.display='none'

# =============================================== SERVICE ACCOUNT ======================================================       
def run_on_service_account_pizdaint(_):
    box_SA.children[0].disabled = True
    INCSCS.layout.display=''
    
    SUBMIT_URL = 'https://bspsa.cineca.it/jobs/pizdaint/' 
    
    command = "unzip "+zfNamePiz+"; cd "+w4.value+"; chmod +rx *.sbatch; ./ipyparallel.sbatch"

    payload = {
        'command': command, 
        'node_number': str(NNODESCSCS.value),
        'core_number': str(NCORESCSCS.value),
        'runtime': str(runtimesa.value),
    }
    
    os.chdir(w4.value)
    job_file = {'file': open(zfNamePiz, 'rb')}
    
    headers = get_oidc_auth()
    headers.update({'Content-Disposition':'attachment;filename=' + zfNamePiz})
    headers.update({'payload':json.dumps(payload)})
    
    r = requests.post(url=SUBMIT_URL, headers=headers, files=job_file)
    if r.status_code == 201:
        job = r.json()
        global jobid
        jobid = job['job_id']
        
        box_SA.children[1].layout.display = ''
    else:
        js = '<script>alert("' + r.content + '")</script>'
        display(HTML(js))
        INCSCS.layout.display = 'none'
        box_SA.children[0].disabled = False
        

def check_on_service_account_pizdaint(_):
    box_SA.children[1].disabled = True
    
    CHECK_JOB_URL = 'https://bspsa.cineca.it/jobs/pizdaint/bsp_pizdaint_01/' + jobid + '/'
    FILES_URL = 'https://bspsa.cineca.it/files/pizdaint/bsp_pizdaint_01/'
    
    headers = get_oidc_auth() 
    
    r = requests.get(url=CHECK_JOB_URL, headers=headers)
    if r.status_code == 200:
        job = r.json()
                
        STNCSCS.layout.display=''
        STNCSCS.value=job['stage']    

        if job['stage'] == 'SUCCESSFUL':
        
            r = requests.get(url=FILES_URL + jobid + '/', headers=headers)
            if r.status_code == 200:
                files_list = r.json()
                clients = get_hbp_service_client()
                storeto_path = os.path.join(get_collab_storage_path(), 'resultsPizdaint-SA')
                if not clients.storage.exists(storeto_path):
                    clients.storage.mkdir(storeto_path)
                storeto = str(os.path.join(storeto_path,'output_'+job['init_date']))
                clients.storage.mkdir(storeto)
                
                for f in files_list:
                    
                    if f == '/stderr' or f == '/stdout' or f == '/output.zip':
                        filename = f[1:]
                        file_type = assign_file_types(f)
                        DOWNLOAD_FILE_URL = FILES_URL + jobid + f + '/'
                        r = requests.get(url=DOWNLOAD_FILE_URL, headers=headers)
                        if r.status_code == 200:
                            with open(filename, 'wb') as local_file:
                                local_file.write(r.content)
                            clients.storage.upload_file(filename, str(os.path.join(storeto, filename)), file_type)
                
                STNCSCS.layout.display = 'none'
                INCSCS.layout.display = 'none'
                checksa_button.layout.display = 'none'
                box_SA.children[1].layout.display = 'none'
                
        else:
            box_SA.children[1].disabled = False

# =======================================================================================================================
        
RunSNSG = Button(description = 'Run NSG simulation', layout=Layout(width='40%'))
RunSNSG.background_color='gainsboro'
RunSNSG.border_color='black'
RunSNSG.width='20%'
RunSNSG.on_click(runNSGifusernameandconfig)
CheckSNSG = Button(description = 'Check NSG simulation', layout=Layout(width='25%'))
CheckSNSG.background_color='gainsboro'
CheckSNSG.border_color='black'
CheckSNSG.on_click(checksimnsg)
buttonSNSG =[RunSNSG, CheckSNSG]
myboxUNSG =HBox(children=buttonSNSG)
myboxUNSG.children[0].layout.display=''
myboxUNSG.children[1].layout.display='none'

RunSCSCS = Button(description = 'Run PizDaint simulation', layout=Layout(width='40%'))
RunSCSCS.background_color='gainsboro'
RunSCSCS.border_color='black'
RunSCSCS.on_click(rununicoreCSCS)
CheckSCSCS = Button(description = 'Check PizDaint simulation', layout=Layout(width='40%'))
CheckSCSCS.background_color='gainsboro'
CheckSCSCS.border_color='black'
CheckSCSCS.on_click(checksimCSCS)
buttonSCSCS =[RunSCSCS, CheckSCSCS]
myboxUCSCS =HBox(children=buttonSCSCS)
myboxUCSCS.children[0].layout.display=''
myboxUCSCS.children[1].layout.display='none'

# ======================================== SERVICE ACCOUNT Accordion Button ============================================
runtimesa = Text(description='Runtime')
runtimesa.value='2.0'
runtimesa.width=190
runsa_button = Button(description='Run Service Account/PizDaint simulation', layout=Layout(width='40%'))
runsa_button.background_color = 'gainsboro'
runsa_button.border_color = 'black'
runsa_button.on_click(run_on_service_account_pizdaint)
checksa_button = Button(description='Check Service Account/PizDaint simulation', layout=Layout(width='40%'))
checksa_button.background_color = 'gainsboro'
checksa_button.border_color = 'black'
checksa_button.layout.display = 'none'
checksa_button.on_click(check_on_service_account_pizdaint)
box_SA = VBox(children=[runsa_button, checksa_button])
# ======================================================================================================================

accordion = widgets.Accordion(children=[widgets.VBox([formMNSG,NNODES,NCORES,RT,\
                                                      STN,IN,BT,myboxUNSG]),\
                                        widgets.VBox([TT,NNODESCSCS,NCORESCSCS,RTCSCS,\
                                                      STNCSCS,INCSCS,BTCSCS,myboxUCSCS],layout=Layout(width='100%')),
                                        widgets.VBox([NNODESCSCS,NCORESCSCS,runtimesa,\
                                                      STNCSCS,INCSCS,BTCSCS,box_SA],layout=Layout(width='100%'))])
accordion.set_title(0, 'NSG')
accordion.set_title(1, 'PizDaint')
accordion.set_title(2, 'Service Account - Pizdaint')        # SERVICE ACCOUNT
accordion

In [ ]:
from IPython.display import HTML
HTML('''
<form action="javascript:code_toggle()">
<input id="toggle-code-button" type="submit" value="Click here to toggle on/off the raw code."></form>
<script>
code_show=true;
console.log(code_show)
$('.widget-area .prompt .close').hide()
function code_toggle() {
 console.log(code_show)
 var list = document.getElementsByClassName('cell code_cell rendered unselected');
 var list_index = []
 for (var i = 0; i < list.length; i++) {
    var curr_des = [];
    curr_wrapper = $(list[i]).find(".output_wrapper");
    curr_area = $(list[i]).find(".output_area");
    if (curr_wrapper.length > 0 && curr_area.length == 0){
    list_index.push(i)
    } else if (curr_wrapper.length == 0 && curr_area.length == 0){
    list_index.push(i)
    };
 }
 console.log(list.length)
 console.log(list_index)
 for (var j = 0; j< list_index.length; j++){
 console.log(list[list_index[j]].style.display)
 }
 if (code_show){
 $('div.input').hide();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "none"
    }
 } else {
 $('div.input').show();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "block"
    }
 }
 code_show = !code_show
 console.log(code_show)

 }
document.getElementById("toggle-code-button").click(); 
</script>
''')